In [1]:
import cv2
import numpy as np
import os
import subprocess


In [2]:
def get_important_scene_times(video_path, threshold_multiplier=3.0, min_scene_length=1.0):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return []

    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_diffs = []
    timestamps = []
    scene_changes = [0.0]  # Start time in seconds

    ret, prev_frame = cap.read()
    if not ret:
        print("Error: Could not read the first frame.")
        return []

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    frame_idx = 1

    while True:
        ret, curr_frame = cap.read()
        if not ret:
            break

        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(curr_gray, prev_gray)
        diff_mean = diff.mean()
        frame_diffs.append(diff_mean)

        prev_gray = curr_gray
        frame_idx += 1

    cap.release()

    # Calculate threshold
    diff_array = np.array(frame_diffs)
    mean_diff = diff_array.mean()
    threshold = mean_diff * threshold_multiplier

    # Detect scene changes
    for i, diff in enumerate(frame_diffs):
        if diff > threshold:
            time = (i + 1) / fps
            scene_changes.append(time)

    scene_changes.append(frame_idx / fps)  # End time of the last scene
    scene_changes = sorted(set(scene_changes))

    # Identify important scenes
    important_scenes = []
    for i in range(len(scene_changes) - 1):
        start_time = scene_changes[i]
        end_time = scene_changes[i + 1]
        scene_duration = end_time - start_time

        if scene_duration >= min_scene_length:
            important_scenes.append((start_time, end_time))

    return important_scenes


In [3]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

def trim_video_moviepy(input_video, scenes, output_video):
    from moviepy.editor import concatenate_videoclips, VideoFileClip

    clips = []
    for start, end in scenes:
        clip = VideoFileClip(input_video).subclip(start, end)
        clips.append(clip)

    final_clip = concatenate_videoclips(clips)
    final_clip.to_videofile(output_video, codec="libx264")


In [4]:
video_path = './shortvid.mp4'
output_video = 'filtered_video.mp4'

# Step 1: Get important scene times
important_scenes = get_important_scene_times(video_path)

# Step 2: Trim the video to keep only important scenes
trim_video_moviepy(video_path, important_scenes, output_video)


Moviepy - Building video filtered_video.mp4.
MoviePy - Writing audio in filtered_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video filtered_video.mp4



Moviepy - Done !
Moviepy - video ready filtered_video.mp4
